In [1]:
import cv2
import openpyxl
import numpy as np
from skimage.transform import resize
import pickle
import pandas as pd
import time
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
EMPTY=True
NOT_EMPTY=False
MODEL = pickle.load(open("C:\\SIH\\paking_counting\\model.p", "rb"))


In [ ]:

def get_parking_spots_bboxes(connected_components):
    (totalLabels, label_ids, values,centroid)=connected_components
    
    slots=[]
    coef=1
    for i in range(1,totalLabels):
        
        x1=int(values[i,cv2.CC_STAT_LEFT]*coef)
        y1=int(values[i,cv2.CC_STAT_TOP]*coef)
        w=int(values[i,cv2.CC_STAT_WIDTH]*coef)
        h=int(values[i,cv2.CC_STAT_HEIGHT]*coef)
        
        slots.append([x1,y1,w,h])
        
    return slots


In [ ]:
def empty_or_not(spot_bgr):
    flat_data=[]
    
    img_resized=resize(spot_bgr,(15,15,3))
    flat_data.append(img_resized.flatten())
    flat_data=np.array(flat_data)
    y_output=MODEL.predict(flat_data)
    if y_output==0:
        return EMPTY
    else:
        return NOT_EMPTY

In [ ]:
def calc_diff(im1,im2):
    return np.abs(np.mean(im1)-np.mean(im2))

In [ ]:
!pip install tensorflow


In [ ]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
import random
import joblib
import time

# Load the machine learning model
loaded_model = joblib.load(r'C:\SIH\Dynamic_pricing\parking_price_prediction_model.pkl')

# Define file paths and parameters
mask_path = r'C:\SIH\paking_counting\mask_1920_1080.png'
video_path = r'C:\SIH\paking_counting\parking_1920_1080.mp4'
output_excel_path = r'C:\SIH\paking_counting\parking_data_count.xlsx'
empty_slots_file = r'C:\SIH\paking_counting\empty_slots.txt'
step = 30
interval = 5
frame_rate = 25
save_interval_frames = interval * frame_rate

# Initialize the video capture
cap = cv2.VideoCapture(video_path)

# Load the parking spot mask
mask = cv2.imread(mask_path, 0)
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CC_STAT_AREA)
spots = get_parking_spots_bboxes(connected_components)  # Assuming you have a function get_parking_spots_bboxes
num_parking_spots = len(spots)
spot_status = [None for _ in spots]
diffs = [None for _ in spots]
previous_frame = None
frame_num = 0

# Initialize the parking data DataFrame
parking_data = pd.DataFrame(columns=['Time', 'Available Spots', 'Occupied Spots', 'Predicted Price', 'Demand'])
time_since_last_save = 0

# Suppress warnings
warnings.filterwarnings("ignore")

previous_occupied_spots = np.zeros(num_parking_spots)
empty_slots = list(range(num_parking_spots))

# Function to update empty slots
def update_empty_slots(spot_index, spot_empty):
    if spot_empty and spot_index not in empty_slots:
        empty_slots.append(spot_index)
    elif not spot_empty and spot_index in empty_slots:
        empty_slots.remove(spot_index)

    with open(empty_slots_file, 'w') as file:
        file.write(','.join(map(str, empty_slots)))

while True:
    ret, frame = cap.read()

    if not ret:
        print("No more frames to read. Exiting the loop.")
        break

    if frame_num % step == 0 and previous_frame is not None:
        for spot_index, spot in enumerate(spots):
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            diffs[spot_index] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

    if frame_num % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_index in arr_:
            spot = spots[spot_index]
            x1, y1, w, h = spot
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            spot_empty = empty_or_not(spot_crop)

            update_empty_slots(spot_index, spot_empty)

            spot_status[spot_index] = spot_empty

    if frame_num % step == 0:
        previous_frame = frame.copy()

    for spot_index, spot in enumerate(spots):
        spot_stat = spot_status[spot_index]
        x1, y1, w, h = spots[spot_index]
        if spot_stat:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    available_spots = len(empty_slots)
    occupied_spots = num_parking_spots - available_spots

    if not np.array_equal(previous_occupied_spots, occupied_spots):
        custom_data = pd.DataFrame({
            'Occupied': [occupied_spots],
            'Timestamp': [current_time],
            'Demand': random.randint(1, 10)
        })

        custom_data['Timestamp'] = pd.to_datetime(custom_data['Timestamp'])

        custom_data['Hour'] = custom_data['Timestamp'].dt.hour
        custom_data['DayOfWeek'] = custom_data['Timestamp'].dt.dayofweek

        custom_data.drop(columns=['Timestamp'], inplace=True)

        custom_data_array = custom_data.to_numpy()

        predicted_price = loaded_model.predict(custom_data_array)

        print(f"Predicted Price: {predicted_price[0]}")

        new_data = pd.DataFrame({'Time': [current_time],
                                 'Available Spots': [available_spots],
                                 'Occupied Spots': [occupied_spots],
                                 'Predicted Price': [predicted_price[0]],
                                 'Demand': [custom_data['Demand'].values[0]]})

        parking_data = pd.concat([new_data, parking_data], ignore_index=True)

        previous_occupied_spots = occupied_spots

    cv2.rectangle(frame, (7, 20), (450, 80), (255, 255, 255), -1)
    cv2.putText(frame, 'Available Spots: {} / {}'.format(str(available_spots), str(num_parking_spots)), (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_num += 1
    time_since_last_save += 1

cap.release()
cv2.destroyAllWindows()

print("Saving data to Excel...")
parking_data.to_excel(output_excel_path, sheet_name='Sheet1', index=False)
print("Data saved successfully.")


In [ ]:
len(empty_slots)

In [ ]:
empty_slots